In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import os 
import logging
import time
spark = SparkSession.builder.master("local[4]") \
    .appName("ETL_CNPJ") \
    .getOrCreate()

In [2]:
current_dir = os.getcwd()
file_log = current_dir + r"/logs/src.log"

# gerando log
logging.basicConfig(level=logging.DEBUG, filename=file_log, format="%(asctime)s - %(levelname)s - %(message)s")

# Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

In [3]:
BASE_ESTABELECIMENTOS = current_dir.replace(r"GitHub\ETL_CNPJ\utilitarios", r"\CNPJ_PROGRAMATICA\ESTABELECIMENTOSCSV/")
ESTABELECIMENTOS = list(filter(lambda x: ".csv" in x, os.listdir(BASE_ESTABELECIMENTOS)))
dados = spark.read.options(delimiter = ";", header=False).csv(f"{BASE_ESTABELECIMENTOS}{ESTABELECIMENTOS[1]}").limit(20)
dados.show(1, vertical=True)

-RECORD 0---------
 _c0  | 07396865  
 _c1  | 0001      
 _c2  | 68        
 _c3  | 1         
 _c4  | null      
 _c5  | 08        
 _c6  | 20170210  
 _c7  | 01        
 _c8  | null      
 _c9  | null      
 _c10 | 20050518  
 _c11 | 1412602   
 _c12 | 1411801   
 _c13 | RUA       
 _c14 | TUCANEIRA 
 _c15 | 30        
 _c16 | null      
 _c17 | DOS LAGOS 
 _c18 | 89136000  
 _c19 | SC        
 _c20 | 8297      
 _c21 | 47        
 _c22 | 33851125  
 _c23 | 47        
 _c24 | 33851125  
 _c25 | 47        
 _c26 | 33851125  
 _c27 | null      
 _c28 | null      
 _c29 | null      
only showing top 1 row



In [4]:
# DEFINE O NOVO NOME DE CADA COLUNA
NOVO_NOME_COLUNAS = {
    "_c0" : "CNPJ_BASE", 
    "_c1" : "CNPJ_ORDEM", 
    "_c2" : "CNPJ_DV", 
    "_c3" : "MATRIZ_FILIAL", 
    "_c4" : "NOME_FANTASIA", 
    "_c5" : "SITUACAO_CADASTRAL", 
    "_c6" : "DATA_SITUACAO_CADASTRAL", 
    "_c7" : "MOTIVO_SITUACAO_CADASTRAL", 
    "_c8" : "CIDADE_EXTERIOR", 
    "_c9" : "PAIS", 
    "_c10" : "DATA_INICIO_ATIVIDADE", 
    "_c11" : "CNAE_PRINCIPAL", 
    "_c12" : "CNAE_SECUNDARIO", 
    "_c13" : "TIPO_LOGRADOURO", 
    "_c14" : "LOGRADOURO", 
    "_c15" : "NUMERO", 
    "_c16" : "COMPLEMENTO", 
    "_c17" : "BAIRRO", 
    "_c18" : "CEP", 
    "_c19" : "UF", 
    "_c20" : "MUNICIPIO", 
    "_c21" : "DDD1", 
    "_c22" : "TELEFONE1", 
    "_c23" : "DDD2", 
    "_c24" : "TELEFONE2", 
    "_c25" : "DDD_FAX", 
    "_c26" : "FAX", 
    "_c27" : "EMAIL", 
    "_c28" : "SITUACAO_ESPECIAL", 
    "_c29" : "DATA_SITUACAO_ESPECIAL"
    }
# DEFINE QUAIS COLUNAS QUEREMOS MANTER E QUAIS QUEREMOS DESCARTAR
COLUNAS_A_MANTER = ['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV' , 'MATRIZ_FILIAL'  , 'NOME_FANTASIA' , 'SITUACAO_CADASTRAL' ,
              'DATA_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  , 'DATA_INICIO_ATIVIDADE'  ,
              'CNAE_PRINCIPAL'  , 'CNAE_SECUNDARIO' , 'TIPO_LOGRADOURO'  , 'LOGRADOURO'  , 'NUMERO'  , 'COMPLEMENTO' ,
              'BAIRRO'  , 'CEP'  , 'UF'  , 'MUNICIPIO'  , 'DDD1'  , 'TELEFONE1'  , 'DDD2'  , 'TELEFONE2'  ,
              'DDD_FAX'  , 'FAX'  , 'EMAIL']


In [5]:
# USA O MÉTODO WITHCOLUMNRENAMED() PARA RENOMEAR AS COLUNAS
for NOME_ANTIGO, NOVO_NOME in NOVO_NOME_COLUNAS.items():
    dados = dados.withColumnRenamed(NOME_ANTIGO, NOVO_NOME)

# USA OS MÉTODOS SELECT() E DROP() PARA DESCARTAR AS COLUNAS QUE NÃO PRECISAMOS
dados = dados.select(COLUNAS_A_MANTER)

# MOSTRA O RESULTADO FINAL COM AS COLUNAS RENOMEADAS E DESCARTADAS
dados.show(1, vertical=True)


-RECORD 0------------------------------
 CNPJ_BASE                 | 07396865  
 CNPJ_ORDEM                | 0001      
 CNPJ_DV                   | 68        
 MATRIZ_FILIAL             | 1         
 NOME_FANTASIA             | null      
 SITUACAO_CADASTRAL        | 08        
 DATA_SITUACAO_CADASTRAL   | 20170210  
 MOTIVO_SITUACAO_CADASTRAL | 01        
 DATA_INICIO_ATIVIDADE     | 20050518  
 CNAE_PRINCIPAL            | 1412602   
 CNAE_SECUNDARIO           | 1411801   
 TIPO_LOGRADOURO           | RUA       
 LOGRADOURO                | TUCANEIRA 
 NUMERO                    | 30        
 COMPLEMENTO               | null      
 BAIRRO                    | DOS LAGOS 
 CEP                       | 89136000  
 UF                        | SC        
 MUNICIPIO                 | 8297      
 DDD1                      | 47        
 TELEFONE1                 | 33851125  
 DDD2                      | 47        
 TELEFONE2                 | 33851125  
 DDD_FAX                   | 47        


In [6]:
dados.show()

+---------+----------+-------+-------------+--------------------+------------------+-----------------------+-------------------------+---------------------+--------------+--------------------+---------------+--------------------+------+--------------------+------------------+--------+---+---------+----+---------+----+---------+-------+--------+--------------------+
|CNPJ_BASE|CNPJ_ORDEM|CNPJ_DV|MATRIZ_FILIAL|       NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_SITUACAO_CADASTRAL|MOTIVO_SITUACAO_CADASTRAL|DATA_INICIO_ATIVIDADE|CNAE_PRINCIPAL|     CNAE_SECUNDARIO|TIPO_LOGRADOURO|          LOGRADOURO|NUMERO|         COMPLEMENTO|            BAIRRO|     CEP| UF|MUNICIPIO|DDD1|TELEFONE1|DDD2|TELEFONE2|DDD_FAX|     FAX|               EMAIL|
+---------+----------+-------+-------------+--------------------+------------------+-----------------------+-------------------------+---------------------+--------------+--------------------+---------------+--------------------+------+--------------------+-------

In [8]:
dados.schema

StructType([StructField('CNPJ_BASE', StringType(), True), StructField('CNPJ_ORDEM', StringType(), True), StructField('CNPJ_DV', StringType(), True), StructField('MATRIZ_FILIAL', StringType(), True), StructField('NOME_FANTASIA', StringType(), True), StructField('SITUACAO_CADASTRAL', StringType(), True), StructField('DATA_SITUACAO_CADASTRAL', StringType(), True), StructField('MOTIVO_SITUACAO_CADASTRAL', StringType(), True), StructField('DATA_INICIO_ATIVIDADE', StringType(), True), StructField('CNAE_PRINCIPAL', StringType(), True), StructField('CNAE_SECUNDARIO', StringType(), True), StructField('TIPO_LOGRADOURO', StringType(), True), StructField('LOGRADOURO', StringType(), True), StructField('NUMERO', StringType(), True), StructField('COMPLEMENTO', StringType(), True), StructField('BAIRRO', StringType(), True), StructField('CEP', StringType(), True), StructField('UF', StringType(), True), StructField('MUNICIPIO', StringType(), True), StructField('DDD1', StringType(), True), StructField('T